# Preprocessing the Queen's House dataset

We need to get the QH input data compatible with what Pointcept will expect.

To that end, let's first take a look at the preprocessed Scannet data. We'll check the test data which lacks the ground truth for now.


In [1]:
import torch
import numpy as np
import laspy
import pandas as pd

In [3]:
# Function to print the contents of the loaded file and details of numpy arrays
def print_structure(data, indent=0):
    for key, value in data.items():
        print('    ' * indent + f'{key}: ', end='')
        if isinstance(value, dict):
            print()
            print_structure(value, indent+1)
        elif isinstance(value, np.ndarray):
            print(f'{type(value)}')
            print('    ' * (indent + 1) + f'Shape: {value.shape}')
            print('    ' * (indent + 1) + f'Dtype: {value.dtype}')
            # Improved presentation of first few elements respecting the shape
            preview_elements = value[:min(5, value.shape[0])] if value.ndim > 1 else value[:min(5, value.size)]
            print('    ' * (indent + 1) + 'First few elements:')
            for elem in preview_elements:
                print('    ' * (indent + 2) + str(elem))
        elif isinstance(value, torch.Tensor):
            print(f'{type(value)}')
            print('    ' * (indent + 1) + f'Shape: {value.shape}')
            print('    ' * (indent + 1) + f'Dtype: {value.dtype}')
            print('    ' * (indent + 1) + 'First few elements:')
            # Ensure tensor is on CPU for numpy conversion and print
            preview_tensor = value[:min(5, value.size(0))] if value.dim() > 1 else value[:min(5, value.numel())]
            if value.requires_grad:
                preview_tensor = preview_tensor.detach()
            preview_tensor = preview_tensor.cpu().numpy()  # Convert to numpy array for easier handling
            for elem in preview_tensor:
                print('    ' * (indent + 2) + str(elem))
        if isinstance(value, str):
            print(type(value), value)
        else:
            print(type(value))

In [6]:
# Path to the .pth file
file_path = '../data/scannet/test/scene0731_00.pth'

# Load the contents of the .pth file
data = torch.load(file_path)



# Call the function to print the structure
print_structure(data)

coord: <class 'numpy.ndarray'>
    Shape: (94940, 3)
    Dtype: float32
    First few elements:
        [0.8089936 1.7126828 0.4325   ]
        [0.80993974 1.7030041  0.4375    ]
        [0.81063616 1.7958345  0.43055147]
        [0.8068326  1.707043   0.48693752]
        [0.81019104 1.7423848  0.40600002]
<class 'numpy.ndarray'>
color: <class 'numpy.ndarray'>
    Shape: (94940, 3)
    Dtype: float32
    First few elements:
        [65. 31. 12.]
        [57. 37. 14.]
        [47. 21.  6.]
        [61. 40. 21.]
        [62. 32. 19.]
<class 'numpy.ndarray'>
scene_id: <class 'str'> scene0731_00
normal: <class 'numpy.ndarray'>
    Shape: (94940, 3)
    Dtype: float32
    First few elements:
        [0.9951747  0.07436273 0.06389181]
        [0.99502224 0.08380412 0.05370668]
        [ 0.9969072  -0.06340786  0.04612521]
        [0.99583614 0.08168959 0.04035537]
        [0.9973334  0.02651416 0.06789085]
<class 'numpy.ndarray'>


So, we have numpy arrays where each element is a len-3 array with:
- coord
- color
- normal
and then a scene id string for the file.

Now, the normals in the Scannet preprocessing script are optional - we may not need them in the final model, so ignore them for now. If we do need them, or we need them to acquire an acceptable performance, we'll have to use the same mesh reconstruction algorithms used in things like Scannet - those are documented on their github and use well-established algorithms, should be feasible to implement for our data.


# QH .las files

Let's get a small snippet that can show what the .las file structure is like


In [5]:
import laspy

def read_las_file(file_path):
    # Open the LAS file
    with laspy.open(file_path) as file:
        for points in file.chunk_iterator(1):
            print("Dimension names:")
            print(list(points.point_format.dimension_names))
            print("Extra dimension names:")
            print(list(points.point_format.extra_dimension_names))
            print(points.point_size)
            
            break
        # Read the point records from the file
        las = file.read()

        # Accessing specific data dimensions
        points = las.points
        x_coordinates = las.x
        y_coordinates = las.y
        z_coordinates = las.z
        red_coordinates = las.red

        # Optionally, access other attributes like intensity, classification, etc.
        intensity = las.intensity
        classifications = las.classification

        # Print some basic information about the LAS file
        print(f"Number of points: {len(points)}")
        print(f"X coordinates: {x_coordinates[:10]}")  # Print first 10 for brevity
        print(f"Y coordinates: {y_coordinates[:10]}")
        print(f"Z coordinates: {z_coordinates[:10]}")
        print(f"red coordinates: {red_coordinates[:10]}")
        print(f"Intensity values: {intensity[:10]}")
        print(f"Classifications: {classifications[:10]}")

# Specify the path to your .las file
file_path = '/data/sdd/training_v2.las'
read_las_file(file_path)

Dimension names:
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'newtax', 'uniclass']
Extra dimension names:
['newtax', 'uniclass']
52
Number of points: 278681989
X coordinates: <ScaledArrayView([538739.77  538738.235 538738.651 538738.941 538739.187 538739.486
 538739.826 538738.029 538738.323 538738.682])>
Y coordinates: <ScaledArrayView([177749.354 177752.248 177751.495 177750.967 177750.523 177749.98
 177749.365 177752.755 177752.22  177751.567])>
Z coordinates: <ScaledArrayView([8.978 8.969 8.987 8.983 8.97  8.971 8.98  8.983 8.976 8.982])>
red coordinates: [46336 46848 46080 46336 46592 46080 44800 43264 43008 42240]
Intensity values: [6966 7224 6708 7224 6192 7224 7740 5418 3870 7740]
Classifications: [0 0 0 0 0 0 0 0 0 0]


So, first thing we need a func that can take this .las file and generate a pytorch state dictionary with it.

Can set up a function to iterate over the requested number of points for testing purposes. If we need to we can extend this script with ground truth info, normal calculations, everything that Pointcept might require.

May also need to look into putting this into the Pointcept package proper as part of the registry, which I'm still figuring out...

## .las converter

So now want some code to convert .las files into whatever format we want. Start with a basic function that can take an input file, and some naive number of points to convert, and outputs a .pth file in a format Pointcept should be able to understand.

In [13]:
def las_to_pth(input_las_path, output_pth_path, num_points):
    # Open the LAS file and read data
    with laspy.open(input_las_path) as file:
        las = file.read()

        # Ensure num_points does not exceed the number of points in the file
        max_points = min(num_points, len(las.points))

        # Create the coord array (n x 3) for coordinates
        coord = np.stack((las.x[:max_points], las.y[:max_points], las.z[:max_points]), axis=-1).astype(np.float32)

        # Check if RGB data exists and create the color array (n x 3) for color data
        if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
            # Convert RGB from uint16 to float32 directly within np.stack
            color = np.stack((las.red[:max_points], las.green[:max_points], las.blue[:max_points]), axis=-1).astype(np.float32)
            # Normalize RGB values if necessary (typical range in LAS is 0 to 65535)
            color /= 256.0  # Normalize to [0, 1] if you prefer to work with normalized colors
        else:
            color = np.zeros((max_points, 3), dtype=np.float32)  # Default to black if no color data available
            print("RGB data not found in LAS file. Defaulting to black for color.")

        # Convert to PyTorch tensors
        tensors = {
            'coord': torch.tensor(coord, dtype=torch.float32),
            'color': torch.tensor(color, dtype=torch.float32),
            'scene_id': "scene9999_00",
        }

    # Save the dictionary as a PyTorch state dictionary (.pth file)
    torch.save(tensors, output_pth_path)
    print(f"Saved {max_points} points to {output_pth_path}")

# Specify the input LAS file, output PTH file path, and number of points
input_las_path = '/data/sdd/training_v2.las'
output_pth_path = 'output_file.pth'
num_points = 1000  # Number of points to process and save

# Call the function
las_to_pth(input_las_path, output_pth_path, num_points)


Saved 1000 points to output_file.pth


Now lets read the output and check it matches.

In [15]:
# Load the contents of the .pth file
data = torch.load(output_pth_path)

# Call the function to print the structure
print_structure(data)

coord: <class 'torch.Tensor'>
    Shape: torch.Size([1000, 3])
    Dtype: torch.float32
    First few elements:
        [5.3873944e+05 1.7774992e+05 8.9729996e+00]
        [5.3873975e+05 1.7774936e+05 8.9779997e+00]
        [5.3873825e+05 1.7775225e+05 8.9689999e+00]
        [5.387386e+05 1.777515e+05 8.987000e+00]
        [5.3873894e+05 1.7775097e+05 8.9829998e+00]
<class 'torch.Tensor'>
color: <class 'torch.Tensor'>
    Shape: torch.Size([1000, 3])
    Dtype: torch.float32
    First few elements:
        [188. 197. 168.]
        [181. 189. 165.]
        [183. 192. 161.]
        [180. 189. 158.]
        [181. 193. 157.]
<class 'torch.Tensor'>
scene_id: <class 'str'> scene9999_00


As I recall from looking at Pointcept's data readers, it can handle torch tensors so even if this doesn't look *exactly* like the Scannet data with it's numpy arrays, this should be *technically* fine as input to Pointcept assuming normals are optional.

As for if the data has the necessary numerical characteristics to be processed properly, we note that the QH data for the x, y, z length scales have large offsets compared to the scannet data for x, y.

Maybe we should get min-max for x, y, z for both the Scannet and QH datasets and compare them?

In [24]:
def las_min_max_ranges(input_las_path):
    # Open the LAS file and read data
    with laspy.open(input_las_path) as file:
        las = file.read()

        # Calculate min and max for x, y, and z
        x_min, x_max = las.x.min(), las.x.max()
        y_min, y_max = las.y.min(), las.y.max()
        z_min, z_max = las.z.min(), las.z.max()

        # Calculate differences
        x_range = x_max - x_min
        y_range = y_max - y_min
        z_range = z_max - z_min

    print(f"X range: {x_min} to {x_max}, Difference: {x_range}")
    print(f"Y range: {y_min} to {y_max}, Difference: {y_range}")
    print(f"Z range: {z_min} to {z_max}, Difference: {z_range}")

# Example usage:
las_min_max_ranges('/data/sdd/training_v2.las')

X range: 538698.605 to 538742.333, Difference: 43.728000000002794
Y range: 177690.766 to 177752.815, Difference: 62.04899999999907
Z range: 6.697 to 29.134, Difference: 22.437


In [29]:
def pth_min_max_ranges_single(input_pth_path):
    # Load the state dictionary from a .pth file
    data = torch.load(input_pth_path)

    # Assuming 'coord' tensor is [n x 3] for x, y, z
    coord = data['coord']

    # Calculate min and max for x, y, and z
    x_min, x_max = coord[:, 0].min().item(), coord[:, 0].max().item()
    y_min, y_max = coord[:, 1].min().item(), coord[:, 1].max().item()
    z_min, z_max = coord[:, 2].min().item(), coord[:, 2].max().item()

    print(f"X range: {x_min} to {x_max}")
    print(f"Y range: {y_min} to {y_max}")
    print(f"Z range: {z_min} to {z_max}")

# Example usage:
pth_min_max_ranges_single('../data/scannet/test/scene0731_00.pth')
print()
pth_min_max_ranges_single('../data/scannet/test/scene0744_00.pth')

X range: 0.56696617603302 to 4.752572059631348
Y range: 0.31187281012535095 to 3.864165782928467
Z range: -0.012044666334986687 to 2.501612901687622

X range: 0.7937601804733276 to 10.793277740478516
Y range: -0.05707528442144394 to 4.980363845825195
Z range: 0.0002449717721901834 to 3.1450295448303223


Let's check all the scannet data min-max ranges at once.

In [21]:
import glob
import os

def pth_min_max_ranges(directory):
    # Recursively find all .pth files in the given directory
    pth_files = glob.glob(os.path.join(directory, '**/*.pth'), recursive=True)

    # Initialize min and max values to None initially
    x_min, x_max = None, None
    y_min, y_max = None, None
    z_min, z_max = None, None

    for file_path in pth_files:
        # Load the state dictionary from a .pth file
        data = torch.load(file_path)

        # Assuming 'coord' tensor is [n x 3] for x, y, z
        coord = data['coord']

        # Calculate min and max for each file and update global min/max
        if x_min is None:
            # Initialize min/max values with the first file's values
            x_min, x_max = coord[:, 0].min(), coord[:, 0].max()
            y_min, y_max = coord[:, 1].min(), coord[:, 1].max()
            z_min, z_max = coord[:, 2].min(), coord[:, 2].max()
        else:
            # Update min/max values based on this file's data
            x_min = min(x_min, coord[:, 0].min())
            x_max = max(x_max, coord[:, 0].max())
            y_min = min(y_min, coord[:, 1].min())
            y_max = max(y_max, coord[:, 1].max())
            z_min = min(z_min, coord[:, 2].min())
            z_max = max(z_max, coord[:, 2].max())

    print(f"Total X range across all files: {x_min.item()} to {x_max.item()}")
    print(f"Total Y range across all files: {y_min.item()} to {y_max.item()}")
    print(f"Total Z range across all files: {z_min.item()} to {z_max.item()}")

# Example usage:
directory_path = '../data/scannet'
pth_min_max_ranges(directory_path)

Total X range across all files: -2.1278457641601562 to 17.116708755493164
Total Y range across all files: -1.4605504274368286 to 18.189441680908203
Total Z range across all files: -0.37873902916908264 to 6.966972827911377


So, between minus a couple to just under 20 on Scannet. Let's check S3DIS too.

In [23]:
pth_min_max_ranges("../data/s3dis")

Total X range across all files: -37.928 to 29.927
Total Y range across all files: -26.078 to 46.05599999999999
Total Z range across all files: -2.6450000000000005 to 6.576


So, broadly similar for S3DIS, plus-minus O(10^2).

I'm unsure how to treat the negative coords in these datasets... might not need to worry about them for now, maybe Pointcept is "smart" enough to just work with data with all positive coords. Seems the majority are positive anyway.

We can modify our .las converter file to remove these large x,y offsets for now.


In [33]:
def las_to_pth(input_las_path, output_pth_path, num_points, norm_z=True):
    # Open the LAS file and read data
    with laspy.open(input_las_path) as file:
        las = file.read()

        # Ensure num_points does not exceed the number of points in the file
        max_points = min(num_points, len(las.points))

        # Read coordinates and adjust them to start at zero
        x_adjusted = las.x[:max_points] - np.min(las.x[:max_points])
        y_adjusted = las.y[:max_points] - np.min(las.y[:max_points])
        
        if norm_z:
            z = las.z[:max_points] - np.min(las.z[:max_points])
        else:
            z = las.z[:max_points]  # Z adjustment is not requested but can be done similarly

        # Create the coord array (n x 3) for adjusted coordinates
        coord = np.stack((x_adjusted, y_adjusted, z), axis=-1).astype(np.float32)

        # Check if RGB data exists and create the color array (n x 3) for color data
        if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
            # Convert RGB from uint16 to float32 directly within np.stack
            color = np.stack((las.red[:max_points], las.green[:max_points], las.blue[:max_points]), axis=-1).astype(np.float32)
            # Normalize RGB values if necessary (typical range in LAS is 0 to 65535)
            color /= 256.0  # Normalize to [0, 1] if you prefer to work with normalized colors
        else:
            color = np.zeros((max_points, 3), dtype=np.float32)  # Default to black if no color data available
            print("RGB data not found in LAS file. Defaulting to black for color.")

        # Convert to PyTorch tensors
        tensors = {
            'coord': torch.tensor(coord, dtype=torch.float32),
            'color': torch.tensor(color, dtype=torch.float32),
            'scene_id': "scene9999_00"
        }

    # Save the dictionary as a PyTorch state dictionary (.pth file)
    torch.save(tensors, output_pth_path)
    print(f"Saved {max_points} points to {output_pth_path}")

In [34]:
# Specify the input LAS file, output PTH file path, and number of points
input_las_path = '/data/sdd/training_v2.las'
output_pth_path = 'output_file_norm.pth'
num_points = 10000  # Number of points to process and save

# Call the function
las_to_pth(input_las_path, output_pth_path, num_points)

Saved 10000 points to output_file_norm.pth


In [35]:
# Load the contents of the .pth file
data = torch.load(output_pth_path)

# Call the function to print the structure
print_structure(data)
print()

pth_min_max_ranges_single(output_pth_path)

coord: <class 'torch.Tensor'>
    Shape: torch.Size([10000, 3])
    Dtype: torch.float32
    First few elements:
        [1.55  0.634 0.061]
        [1.868 0.059 0.066]
        [0.333 2.953 0.057]
        [0.749 2.2   0.075]
        [1.039 1.672 0.071]
<class 'torch.Tensor'>
color: <class 'torch.Tensor'>
    Shape: torch.Size([10000, 3])
    Dtype: torch.float32
    First few elements:
        [188. 197. 168.]
        [181. 189. 165.]
        [183. 192. 161.]
        [180. 189. 158.]
        [181. 193. 157.]
<class 'torch.Tensor'>
scene_id: <class 'str'> scene9999_00

X range: 0.0 to 4.431000232696533
Y range: 0.0 to 3.5199999809265137
Z range: 0.0 to 0.1770000010728836


I note that this is a very very crude approach, we'll very likely have to do a smarter coordinate normalisation in our preprocessing, even if it seems like Pointcept handles a lot of normalisation inside the "black box" itself and not in preprocessing.

# Plugging this data into Pointcept

So now we have converted a chunk of QH data into something that looks superficially reasonable for a scannet/s3dis dataset, can we just plug this into a Pointcept config file and run some inference with it?

Time to dig into the config file format and see if we can configure a job that just runs inference. If not we'll maybe need to spoof some ground-truth values.

Let's have a look at what the ground truth values look like in the Scannet processed dataset.

In [36]:
file_path = '../data/scannet/val/scene0011_00.pth'
data = torch.load(file_path)
print_structure(data)

coord: <class 'numpy.ndarray'>
    Shape: (237360, 3)
    Dtype: float32
    First few elements:
        [2.5091114  0.4083811  0.14877559]
        [2.5156426  0.4059527  0.14168811]
        [2.5073788  0.4145141  0.14327997]
        [2.5004458  0.40534744 0.1462612 ]
        [2.5201747  0.47421882 0.15652634]
<class 'numpy.ndarray'>
color: <class 'numpy.ndarray'>
    Shape: (237360, 3)
    Dtype: float32
    First few elements:
        [35. 33. 38.]
        [34. 32. 39.]
        [40. 35. 43.]
        [35. 33. 38.]
        [31. 30. 31.]
<class 'numpy.ndarray'>
scene_id: <class 'str'> scene0011_00
normal: <class 'numpy.ndarray'>
    Shape: (237360, 3)
    Dtype: float32
    First few elements:
        [0.19109516 0.92176497 0.3371149 ]
        [0.35799062 0.9311391  0.06885417]
        [ 0.59007823 -0.42843354  0.684249  ]
        [0.2563957  0.11805225 0.9592446 ]
        [ 0.819725   -0.5543586   0.14370042]
<class 'numpy.ndarray'>
semantic_gt20: <class 'numpy.ndarray'>
    Shape: (23

So we have the ground truth for Scannet20, 200, and an "instance_gt" which might be a general label? Let's check the scannet preprocessing script to see what this gt actually is.

```python
 # Generate new labels
        semantic_gt20 = np.ones((vertices.shape[0])) * IGNORE_INDEX
        semantic_gt200 = np.ones((vertices.shape[0])) * IGNORE_INDEX
        instance_ids = np.ones((vertices.shape[0])) * IGNORE_INDEX
        for group in seg_groups:
            point_idx, label_id20, label_id200 = point_indices_from_group(
                seg_indices, group, labels_pd
            )

            semantic_gt20[point_idx] = label_id20
            semantic_gt200[point_idx] = label_id200
            instance_ids[point_idx] = group["id"]

        semantic_gt20 = semantic_gt20.astype(int)
        semantic_gt200 = semantic_gt200.astype(int)
        instance_ids = instance_ids.astype(int)

        save_dict["semantic_gt20"] = semantic_gt20
        save_dict["semantic_gt200"] = semantic_gt200
        save_dict["instance_gt"] = instance_ids

```

## Normal spoofing

So turns out we need normals for a strict scannet set. For the time being I'm just going to spoof some normal vectors and run with it, then we'll introduce our own datasets and see if normals are required or not.



In [37]:
def las_to_pth(input_las_path, output_pth_path, num_points):
    # Open the LAS file and read data
    with laspy.open(input_las_path) as file:
        las = file.read()

        # Ensure num_points does not exceed the number of points in the file
        max_points = min(num_points, len(las.points))

        # Create the coord array (n x 3) for coordinates
        coord = np.stack((las.x[:max_points], las.y[:max_points], las.z[:max_points]), axis=-1).astype(np.float32)

        # Check if RGB data exists and create the color array (n x 3) for color data
        if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
            # Convert RGB from uint16 to float32 directly within np.stack
            color = np.stack((las.red[:max_points], las.green[:max_points], las.blue[:max_points]), axis=-1).astype(np.float32)
            # Normalize RGB values if necessary (typical range in LAS is 0 to 65535)
            color /= 256.0  # Normalize to [0, 1] if you prefer to work with normalized colors
        else:
            color = np.zeros((max_points, 3), dtype=np.float32)  # Default to black if no color data available
            print("RGB data not found in LAS file. Defaulting to black for color.")

        # Generate random normals and normalize them
        normal = np.random.rand(max_points, 3).astype(np.float32)
        norm = np.linalg.norm(normal, axis=1, keepdims=True)
        normal /= norm  # Normalize to unit vectors

        # Convert to PyTorch tensors
        tensors = {
            'coord': torch.tensor(coord, dtype=torch.float32),
            'color': torch.tensor(color, dtype=torch.float32),
            'normal': torch.tensor(normal, dtype=torch.float32),
            'scene_id': "scene9999_00"  # Add a top-level string object with a dummy value
        }

    # Save the dictionary as a PyTorch state dictionary (.pth file)
    torch.save(tensors, output_pth_path)
    print(f"Saved {max_points} points to {output_pth_path}")

In [38]:
# Specify the input LAS file, output PTH file path, and number of points
input_las_path = '/data/sdd/training_v2.las'
output_pth_path = 'output_file_norm.pth'
num_points = 1000000  # Number of points to process and save

# Call the function
las_to_pth(input_las_path, output_pth_path, num_points)

Saved 10000 points to output_file_norm.pth


In [39]:
data = torch.load(output_pth_path)
print_structure(data)

coord: <class 'torch.Tensor'>
    Shape: torch.Size([10000, 3])
    Dtype: torch.float32
    First few elements:
        [5.3873944e+05 1.7774992e+05 8.9729996e+00]
        [5.3873975e+05 1.7774936e+05 8.9779997e+00]
        [5.3873825e+05 1.7775225e+05 8.9689999e+00]
        [5.387386e+05 1.777515e+05 8.987000e+00]
        [5.3873894e+05 1.7775097e+05 8.9829998e+00]
<class 'torch.Tensor'>
color: <class 'torch.Tensor'>
    Shape: torch.Size([10000, 3])
    Dtype: torch.float32
    First few elements:
        [188. 197. 168.]
        [181. 189. 165.]
        [183. 192. 161.]
        [180. 189. 158.]
        [181. 193. 157.]
<class 'torch.Tensor'>
normal: <class 'torch.Tensor'>
    Shape: torch.Size([10000, 3])
    Dtype: torch.float32
    First few elements:
        [0.47011682 0.23847726 0.8497757 ]
        [0.20806971 0.5027224  0.8390335 ]
        [0.5014625  0.17515276 0.84726435]
        [0.7702582  0.5183476  0.37150773]
        [0.59888804 0.3688299  0.71084285]
<class 'torch.Ten

In [9]:
def las_to_np_pth(input_las_path, output_pth_path, num_points, scene_id, spoof_normal=True, spoof_gt=True):
    # Open the LAS file and read data
    with laspy.open(input_las_path) as file:
        las = file.read()

        # Ensure num_points does not exceed the number of points in the file
        max_points = min(num_points, len(las.points))

        # Read coordinates and adjust them to start at zero
        x_adjusted = las.x[:max_points] - np.min(las.x[:max_points])
        y_adjusted = las.y[:max_points] - np.min(las.y[:max_points])
        z_adjusted = las.z[:max_points] - np.min(las.z[:max_points])
        
        # Create the coord array (n x 3) for coordinates
        # coord = np.stack((las.x[:max_points], las.y[:max_points], las.z[:max_points]), axis=-1).astype(np.float32)
        coord = np.stack((x_adjusted, y_adjusted, z_adjusted), axis=-1).astype(np.float32)
        
        # Check if RGB data exists and create the color array (n x 3) for color data
        if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
            color = np.stack((las.red[:max_points], las.green[:max_points], las.blue[:max_points]), axis=-1).astype(np.float32)
            color /= 256.0  # Normalize to [0, 1] if you prefer to work with normalized colors
        else:
            color = np.zeros((max_points, 3), dtype=np.float32)  # Default to black if no color data available
            print("RGB data not found in LAS file. Defaulting to black for color.")

        # Generate random normals and normalize them
        normal = np.random.rand(max_points, 3).astype(np.float32)
        norm = np.linalg.norm(normal, axis=1, keepdims=True)
        normal /= norm

        # Save numpy arrays into a dictionary and then to a .pth file
        data = {
            'coord': coord,
            'color': color,
            'scene_id': scene_id,
        }

        if spoof_normal:
            # Generate random normals and normalize them
            normal = np.random.rand(max_points, 3).astype(np.float32)
            norm = np.linalg.norm(normal, axis=1, keepdims=True)
            normal /= norm
            data['normal'] = normal

        if spoof_gt:
            semantic_gt20 = np.random.randint(0, 20, size=max_points, dtype=np.int64)
            data['semantic_gt20'] = semantic_gt20
        
        # Save the dictionary as a .pth file
        torch.save(data, output_pth_path)
        print(f"Saved {max_points} points to {output_pth_path}")

In [10]:
# input_las_path = '/data/sdd/training_v2.las'
input_las_path = '/data/sdd/training_v2_subsampled_centroid_0.05.las'
output_pth_path = 'output_file_norm.pth'
las_to_np_pth(input_las_path, output_pth_path, int(1e9), "scene0011_00", spoof_normal=True)

Saved 2012921 points to output_file_norm.pth


In [11]:
data = torch.load(output_pth_path)
print_structure(data)

coord: <class 'numpy.ndarray'>
    Shape: (2012921, 3)
    Dtype: float32
    First few elements:
        [13.32   0.015  4.674]
        [13.375  0.016  4.678]
        [1.3419e+01 6.0000e-03 4.6690e+00]
        [13.422  0.015  4.723]
        [1.3437e+01 1.1000e-02 4.7550e+00]
<class 'numpy.ndarray'>
color: <class 'numpy.ndarray'>
    Shape: (2012921, 3)
    Dtype: float32
    First few elements:
        [165. 164. 146.]
        [166. 161. 142.]
        [68. 70. 57.]
        [91. 92. 86.]
        [179. 178. 176.]
<class 'numpy.ndarray'>
scene_id: <class 'str'> scene0011_00
normal: <class 'numpy.ndarray'>
    Shape: (2012921, 3)
    Dtype: float32
    First few elements:
        [0.14092363 0.35768595 0.92314744]
        [0.5125484  0.85069805 0.11664961]
        [0.69072914 0.722919   0.01677639]
        [0.6820378  0.00235367 0.7313131 ]
        [0.7649099  0.40326482 0.502285  ]
<class 'numpy.ndarray'>
semantic_gt20: <class 'numpy.ndarray'>
    Shape: (2012921,)
    Dtype: int64
    F

In [21]:
def print_npy_file_contents(npy_file_path):
    # Load the numpy array from the .npy file
    array = np.load(npy_file_path)
    
    # Print type, shape, and data type of the array
    print("Array:", type(array))
    print("    Shape:", array.shape)
    print("    Dtype:", array.dtype)
    
    # Print first few elements of the array
    print("    First few elements:")
    # Limit the number of elements to print, for example, print the first 5 elements
    preview_count = min(5, array.size)  # Use min to avoid index error for small arrays
    for index in range(preview_count):
        print("        ", array.flat[index])  # Use flat indexing for simplicity

# Example usage
npy_file_path = '../exp/test/semseg-pt-v3m1-1-ppt-extreme/result/scene0011_00_pred.npy'
print_npy_file_contents(npy_file_path)

Array: <class 'numpy.ndarray'>
    Shape: (10000,)
    Dtype: int64
    First few elements:
         1
         1
         1
         1
         1


In [11]:
import pandas as pd

def array_to_series_and_show_values(npy_file_path):
    # Load the numpy array from the .npy file
    array = np.load(npy_file_path)
    
    # Convert the numpy array to a pandas Series
    series = pd.Series(array)
    
    # Get unique values present in the Series
    unique_values = series.unique()
    
    # Print the unique values
    print("Unique values in the array:", unique_values)

    # Optionally, return the Series if you want to use it later
    return series
npy_file_path = '../exp/test/semseg-pt-v3m1-1-ppt-extreme/result/scene0011_00_pred.npy'
series = array_to_series_and_show_values(npy_file_path)

Unique values in the array: [1]


In [48]:
series

0       1
1       1
2       1
3       1
4       1
       ..
9995    1
9996    1
9997    1
9998    1
9999    1
Length: 10000, dtype: int64

In [7]:
file_path = "/home/sogilvy/Pointcept/notebooks/output_voxels/voxel_0_1_0.pth"
# Load the contents of the .pth file
data = torch.load(file_path)
print_structure(data)

coord: <class 'numpy.ndarray'>
    Shape: (10178, 3)
    Dtype: float32
    First few elements:
        [ 0.028 22.564  3.568]
        [ 0.024 22.561  3.624]
        [ 0.035 22.561  3.677]
        [1.9000e-02 2.2551e+01 3.8380e+00]
        [ 0.023 22.571  4.48 ]
<class 'numpy.ndarray'>
color: <class 'numpy.ndarray'>
    Shape: (10178, 3)
    Dtype: float32
    First few elements:
        [163. 163. 171.]
        [173. 168. 174.]
        [123. 132. 131.]
        [129. 133. 142.]
        [185. 185. 175.]
<class 'numpy.ndarray'>
normal: <class 'numpy.ndarray'>
    Shape: (10178, 3)
    Dtype: float32
    First few elements:
        [0.8561319  0.40729362 0.3180411 ]
        [0.5704726  0.38740727 0.7242076 ]
        [0.9515962  0.29082394 0.09942879]
        [0.36304554 0.8517998  0.37767047]
        [0.59822357 0.79330564 0.11311374]
<class 'numpy.ndarray'>
semantic_gt20: <class 'numpy.ndarray'>
    Shape: (10178,)
    Dtype: int64
    First few elements:
        8
        5
        12
 